<a href="https://colab.research.google.com/github/JunyueLiu/wgan/blob/master/wgan_toy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import TensorFlow and other libraries

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
try:
  # %tensorflow_version 只在 Colab 中使用。
  %tensorflow_version 2.x
except Exception:
  pass

In [0]:
import tensorflow as tf

In [0]:
tf.__version__

In [0]:
# 用于生成 GIF 图片
!pip install imageio

In [0]:
import glob
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os
import PIL
from tensorflow.keras import layers
import time
import random
import sklearn.datasets
from IPython import display

In [0]:
MODE = 'wgan-gp' # wgan or wgan-gp
DATASET = '8gaussians' # 8gaussians, 25gaussians, swissroll
DIM = 512 # Model dimensionality
FIXED_GENERATOR = False # whether to hold the generator fixed at real data plus
                        # Gaussian noise, as in the plots in the paper
LAMBDA = .1 # Smaller lambda makes things faster for toy tasks, but isn't
            # necessary if you increase CRITIC_ITERS enough
CRITIC_ITERS = 5 # How many critic iterations per generator iteration
BATCH_SIZE = 256 # Batch size
ITERS = 100000 # how many generator iterations to train for

In [0]:
def inf_train_gen():
    if DATASET == '25gaussians':
    
        dataset = []
        for i in range(100000/25):
            for x in range(-2, 3):
                for y in range(-2, 3):
                    point = np.random.randn(2)*0.05
                    point[0] += 2*x
                    point[1] += 2*y
                    dataset.append(point)
        dataset = np.array(dataset, dtype='float32')
        np.random.shuffle(dataset)
        dataset /= 2.828 # stdev
        while True:
            for i in range(len(dataset)/BATCH_SIZE):
                yield dataset[i*BATCH_SIZE:(i+1)*BATCH_SIZE]

    elif DATASET == 'swissroll':

        while True:
            data = sklearn.datasets.make_swiss_roll(
                n_samples=BATCH_SIZE, 
                noise=0.25
            )[0]
            data = data.astype('float32')[:, [0, 2]]
            data /= 7.5 # stdev plus a little
            yield data

    elif DATASET == '8gaussians':
    
        scale = 2.
        centers = [
            (1,0),
            (-1,0),
            (0,1),
            (0,-1),
            (1./np.sqrt(2), 1./np.sqrt(2)),
            (1./np.sqrt(2), -1./np.sqrt(2)),
            (-1./np.sqrt(2), 1./np.sqrt(2)),
            (-1./np.sqrt(2), -1./np.sqrt(2))
        ]
        centers = [(scale*x,scale*y) for x,y in centers]
        while True:
            dataset = []
            for i in range(BATCH_SIZE):
                point = np.random.randn(2)*.02
                center = random.choice(centers)
                point[0] += center[0]
                point[1] += center[1]
                dataset.append(point)
            dataset = np.array(dataset, dtype='float32')
            dataset /= 1.414 # stdev
            yield dataset

In [0]:
# def Linear( 
#         input_dim, 
#         output_dim, 
#         inputs,
#         biases=True,
#         initialization=None,
#         weightnorm=None,
#         gain=1.
#         ):
#     """
#     initialization: None, `lecun`, 'glorot', `he`, 'glorot_he', `orthogonal`, `("uniform", range)`
#     """

#       def uniform(stdev, size):
#           if _weights_stdev is not None:
#               stdev = _weights_stdev
#           return np.random.uniform(
#               low=-stdev * np.sqrt(3),
#               high=stdev * np.sqrt(3),
#               size=size
#           ).astype('float32')

#       if initialization == 'lecun':# and input_dim != output_dim):
#           # disabling orth. init for now because it's too slow
#           weight_values = uniform(
#               np.sqrt(1./input_dim),
#               (input_dim, output_dim)
#           )

#       elif initialization == 'glorot' or (initialization == None):

#           weight_values = uniform(
#               np.sqrt(2./(input_dim+output_dim)),
#               (input_dim, output_dim)
#           )

#       elif initialization == 'he':

#           weight_values = uniform(
#               np.sqrt(2./input_dim),
#               (input_dim, output_dim)
#           )

#       elif initialization == 'glorot_he':

#           weight_values = uniform(
#               np.sqrt(4./(input_dim+output_dim)),
#               (input_dim, output_dim)
#           )

#       elif initialization == 'orthogonal' or \
#           (initialization == None and input_dim == output_dim):
          
#           # From lasagne
#           def sample(shape):
#               if len(shape) < 2:
#                   raise RuntimeError("Only shapes of length 2 or more are "
#                                       "supported.")
#               flat_shape = (shape[0], np.prod(shape[1:]))
#                 # TODO: why normal and not uniform?
#               a = np.random.normal(0.0, 1.0, flat_shape)
#               u, _, v = np.linalg.svd(a, full_matrices=False)
#               # pick the one with the correct shape
#               q = u if u.shape == flat_shape else v
#               q = q.reshape(shape)
#               return q.astype('float32')
#           weight_values = sample((input_dim, output_dim))
      
#       elif initialization[0] == 'uniform':
      
#           weight_values = np.random.uniform(
#               low=-initialization[1],
#               high=initialization[1],
#               size=(input_dim, output_dim)
#           ).astype('float32')

#       else:

#           raise Exception('Invalid initialization!')

#       weight_values *= gain

#       weight = weight_values

#       if weightnorm==None:
#         weightnorm = _default_weightnorm
#       if weightnorm:
#         norm_values = np.sqrt(np.sum(np.square(weight_values), axis=0))
#           # norm_values = np.linalg.norm(weight_values, axis=0)

#         target_norms = norm_values
#         norms = tf.sqrt(tf.reduce_sum(tf.square(weight), reduction_indices=[0]))
#         weight = weight * (target_norms / norms)

#         # if 'Discriminator' in name:
#         #     print "WARNING weight constraint on {}".format(name)
#         #     weight = tf.nn.softsign(10.*weight)*.1

#       # if inputs.get_shape().ndims == 2:
#       #     result = tf.matmul(inputs, weight)
#       # else:
#       #     reshaped_inputs = tf.reshape(inputs, [-1, input_dim])
#       #     result = tf.matmul(reshaped_inputs, weight)
#       #     result = tf.reshape(result, tf.pack(tf.unpack(tf.shape(inputs))[:-1] + [output_dim]))

#       # if biases:
#       #     result = tf.nn.bias_add(
#       #         result,
#       #         lib.param(
#       #             name + '.b',
#       #             np.zeros((output_dim,), dtype='float32')
#       #         )
#       #     )
#     result = layers.Dense(units=output_dim, use_bias=biases, )
#     return result

In [0]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(512, use_bias=True,input_shape=(2,), kernel_initializer=tf.keras.initializers.he_uniform()))
    model.add(layers.ReLU())
    model.add(layers.Dense(512, kernel_initializer=tf.keras.initializers.he_uniform()))
    model.add(layers.ReLU())
    model.add(layers.Dense(512, kernel_initializer=tf.keras.initializers.he_uniform()))
    model.add(layers.ReLU())
    model.add(layers.Dense(2))
    return model

使用（尚未训练的）生成器创建一张图片。

In [0]:
generator_test = make_generator_model()
noise = tf.random.normal([5,2])
generated_distribution = generator_test(noise, training=False)
# plt.imshow(generated_image, cmap='gray')
n_distribution = generated_distribution.numpy()
plt.scatter(n_distribution[:,0], n_distribution[:,1])


### 判别器

In [0]:
def make_discriminator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(512, use_bias=True, input_shape=(2,), kernel_initializer=tf.keras.initializers.he_uniform()))
    model.add(layers.ReLU())
    model.add(layers.Dense(512, kernel_initializer=tf.keras.initializers.he_uniform()))
    model.add(layers.ReLU())
    model.add(layers.Dense(512, kernel_initializer=tf.keras.initializers.he_uniform()))
    model.add(layers.ReLU())
    model.add(layers.Dense(1, kernel_initializer=tf.keras.initializers.he_uniform()))
    return model

使用（尚未训练的）判别器

In [0]:
discriminator = make_discriminator_model()
decision = tf.reshape(discriminator(generated_distribution),[-1])
decision

## 定义损失函数和优化器

为两个模型定义损失函数和优化器。

In [0]:
def discriminator_loss(real_output, fake_output):
    disc_cost = tf.reduce_mean(fake_output) - tf.reduce_mean(real_output)
    return disc_cost

### 生成器损失

生成器损失量化其欺骗判别器的能力。直观来讲，如果生成器表现良好，判别器将会把伪造图片判断为真实图片（或 1）。这里我们将把判别器在生成图片上的判断结果与一个值全为 1 的数组进行对比。

In [0]:
def generator_loss(fake_output):
    return -tf.reduce_mean(fake_output)

由于我们需要分别训练两个网络，判别器和生成器的优化器是不同的。

In [0]:
generator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, 
        beta_1=0.5, 
        beta_2=0.9)
discriminator_optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4, 
        beta_1=0.5, 
        beta_2=0.9)

## 定义训练循环



In [0]:
EPOCHS = 100000
noise_dim = 2
num_examples_to_generate = 256


# 我们将重复使用该种子（因此在动画 GIF 中更容易可视化进度）
seed = tf.random.normal([num_examples_to_generate, noise_dim])

In [0]:
frame_index = [0]
def generate_image(true_dist, generator,discriminator):
    """
    Generates and saves a plot of the true distribution, the generator, and the
    critic.
    """
    N_POINTS = 128
    RANGE = 3

    points = np.zeros((N_POINTS, N_POINTS, 2), dtype='float32')
    points[:,:,0] = np.linspace(-RANGE, RANGE, N_POINTS)[:,None]
    points[:,:,1] = np.linspace(-RANGE, RANGE, N_POINTS)[None,:]
    points = points.reshape((-1,2))
    fake_data = generator(points, training=False).numpy()
    disc_real = discriminator(true_dist,training=False).numpy()
    disc_map = discriminator(points, training=False).numpy()
    # samples, disc_map = session.run(
    #     [fake_data, disc_real], 
    #     feed_dict={real_data:points}
    # )
    # disc_map = session.run(disc_real, feed_dict={real_data:points})

    plt.clf()

    x = y = np.linspace(-RANGE, RANGE, N_POINTS)
    plt.contour(x,y,disc_map.reshape((len(x), len(y))).transpose())

    plt.scatter(true_dist[:, 0], true_dist[:, 1], c='orange',  marker='+')
    plt.scatter(fake_data[:, 0], fake_data[:, 1], c='green', marker='+')
    
    plt.savefig('frame'+str(frame_index[0])+'.png')
    plt.show()
    frame_index[0] += 1

In [0]:
# 注意 `tf.function` 的使用
# 该注解使函数被“编译”
generator = make_generator_model()
discriminator = make_discriminator_model()

@tf.function
def train_step(real_data):
    # initial variables for weight penalty
    alpha = tf.random.uniform(
          shape=[BATCH_SIZE,1], 
          minval=0.,
          maxval=1.,
        ) 
    noise = tf.random.normal([BATCH_SIZE, noise_dim])
    # interpolates = (alpha * real_data + ((1-alpha) * noise))

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
      generated_data = generator(noise, training=True)
      fake_output = tf.reshape(discriminator(generated_data, training=True),[-1])
      gen_loss = generator_loss(fake_output)
      interpolates = (alpha * real_data + ((1-alpha) * generated_data))
      with tf.GradientTape() as tape:
        tape.watch(interpolates)
        real_output_0 = discriminator(real_data, training=True)
        real_output = tf.reshape(real_output_0,[-1])
        disc_interpolates = discriminator(interpolates, training=False)
      gradients = tape.gradient(disc_interpolates, interpolates)
      # print(gradients)
      slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients)))
      gradient_penalty = tf.reduce_mean((slopes-1)**2)

      # discriminate loss penalty
      disc_loss = discriminator_loss(real_output, fake_output)+ LAMBDA*gradient_penalty
    # get the graients
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [0]:
def train(epochs):
    gen = inf_train_gen()
    for epoch in range(epochs):
      for i in range(CRITIC_ITERS):
        _data = next(gen)
        _data = tf.convert_to_tensor(_data, dtype=tf.float32)
        train_step(_data)
      # 每 15 个 epoch 保存一次模型
      if (epoch + 1) % 15 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)
      if epoch % 100 == 99:
        generate_image(_data, generator, discriminator)

**生成与保存图片**



In [0]:
%%time
train(EPOCHS)

恢复最新的检查点。

In [0]:
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## 创建 GIF


In [0]:
# 使用 epoch 数生成单张图片
# def display_image(epoch_no):
#   return PIL.Image.open('image_at_epoch_{:04d}.png'.format(epoch_no))

In [0]:
# display_image(EPOCHS)

使用训练过程中生成的图片通过 `imageio` 生成动态 gif 

In [0]:
anim_file = 'wgan_swiss_2.gif'

with imageio.get_writer(anim_file, mode='I') as writer:
  filenames = glob.glob('frame*.png')
  filenames = sorted(filenames)
  last = -1
  for i,filename in enumerate(filenames):
    if i % 10 !=0:
      continue
    image = imageio.imread(filename)
    writer.append_data(image)
  image = imageio.imread(filename)
  writer.append_data(image)

import IPython
if IPython.version_info > (6,2,0,''):
  display.Image(filename=anim_file)

如果您正在使用 Colab，您可以通过如下代码下载动画：

In [0]:
try:
  from google.colab import files
except ImportError:
   pass
else:
  files.download(anim_file)

## 下一步


本教程展示了实现和训练 GAN 模型所需的全部必要代码。接下来，您可能想尝试其他数据集，例如大规模名人面部属性（CelebA）数据集 [在 Kaggle 上获取](https://www.kaggle.com/jessicali9530/celeba-dataset/home)。要了解更多关于 GANs 的信息，我们推荐参阅 [NIPS 2016 教程： 生成对抗网络](https://arxiv.org/abs/1701.00160)。